In [1]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from scipy.sparse import coo_matrix
from pyomo.contrib.pynumero.interfaces.external_grey_box import ExternalGreyBoxModel
from pyomo.contrib.pynumero.interfaces.external_grey_box import ExternalGreyBoxBlock

In [2]:
from greybox_generalize import LogDetModel # import grey-box module

In [3]:
def build_model_external(m):
    '''Set up grey-box module
    '''
    ex_model = LogDetModel(num_para=5)
    m.egb = ExternalGreyBoxBlock()
    m.egb.set_external_model(ex_model)

In [4]:
# FIM units
FIM_lists = {
    0: [[10, 20], [20, 50]],
    1: [[5, 10], [10, 1]],
    2: [[50, -1], [-1, 10]]
}

In [8]:
def compute_FIM(FIM_list, mip=True):
    
    m = pyo.ConcreteModel()
    Np = 2
    Ny = 3
    
    m.Ny = pyo.Set(initialize=range(Ny))
    m.DimFIM = pyo.Set(initialize=range(Np))
    
    def identity(m, a, b):
        return 1 if a==b else 0
    
    # y are decision variables
    if mip: # mixed integer problem 
        m.y = pyo.Var(m.Ny, bounds=(0,1), initialize=0, within=pyo.Binary)
    else: # relax the MIP problem
        m.y = pyo.Var(m.Ny, bounds=(0,1), initialize=0.5, within=pyo.NonNegativeReals)
        
    # grey-box model inputs
    m.TotalFIM = pyo.Var(m.DimFIM, m.DimFIM, initialize=identity, within=pyo.Reals)
    
    # compute Total FIM with: M = y1*M1 + y2*M2 + y3*M3
    def compute_fim_y(m,i,j):
        return m.TotalFIM[i,j] == sum(m.y[a]*FIM_list[a][i][j] for a in m.Ny)
    
    m.fim_compute_y = pyo.Constraint(m.DimFIM, m.DimFIM, rule=compute_fim_y)
    
    # create grey-box block
    def _model_i(b):
        build_model_external(b)
    m.my_block = pyo.Block(rule=_model_i)

    # fix grey-box inputs 
    for i in m.DimFIM:
        for j in range(i, Np):
            def eq_fim(m):
                return m.TotalFIM[i,j] == m.my_block.egb.inputs['ele_'+str(i)+'_'+str(j)]

            con_name = "con"+str(i)+str(j)
            m.add_component(con_name, pyo.Constraint(expr=eq_fim))

    # add objective to maximize log det
    m.Obj = pyo.Objective(expr=m.my_block.egb.outputs['log_det'], sense=pyo.maximize)
    
    return m 

In [9]:
mod = compute_FIM(FIM_lists, mip=False)

solver = pyo.SolverFactory('cyipopt')
solver.config.options['hessian_approximation'] = 'limited-memory' 
additional_options={'max_iter':3000}

for k,v in additional_options.items():
    solver.config.options[k] = v
results = solver.solve(mod, tee=True)





 Consider M =
 [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
   logdet =  0.0 


 Consider M =
 [[1.12436656 0.0742421  0.         0.         0.        ]
 [0.0742421  1.06613987 0.         0.         0.        ]
 [0.         0.         1.18803472 0.         0.        ]
 [0.         0.         0.         1.18803472 0.        ]
 [0.         0.         0.         0.         1.18803472]]
   logdet =  0.6935569841782372 


 Consider M =
 [[11.35545838  2.722986    0.          0.          0.        ]
 [ 2.722986    8.27610427  0.          0.          0.        ]
 [ 0.          0.          2.95332019  0.          0.        ]
 [ 0.          0.          0.          2.95332019  0.        ]
 [ 0.          0.          0.          0.          2.95332019]]
   logdet =  7.709677624739804 


 Consider M =
 [[8.83589189 1.79082474 0.         0.         0.        ]
 [1.79082474 5.98074195 0.         0.         0.        ]
 [0.         0.         6.62529805 0


 Consider M =
 [[6.00000006e+01 1.90000001e+01 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [1.90000001e+01 6.00000006e+01 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 9.64411356e+09 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 9.64411356e+09
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  9.64411356e+09]]
   logdet =  77.05186065708726 


 Consider M =
 [[6.00000006e+01 1.90000001e+01 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [1.90000001e+01 6.00000006e+01 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.56087153e+10 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.56087153e+10
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.56087153e+10]]
   logdet =  78.4963057433494 


 Consider M =
 [[6.00000006e+01 1.90000001e+01 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [1.9

ValueError: Numeric value `-8.3841153059088e-09` (float64) is not in domain NonNegativeReals for Var y[1]

In [10]:
# cyipopt solves the relax problem
print(pyo.value(mod.y[0]), pyo.value(mod.y[1]), pyo.value(mod.y[2]))

1.0000000099886204 0.5 0.5


In [11]:
# solve the MIP problem

mod = compute_FIM(FIM_lists, mip=True)
solver = pyo.SolverFactory('mindtpy')
results = solver.solve(mod, mip_solver='gurobi_persistent', nlp_solver='cyipopt', tee=True, integer_tolerance=0.5)
solver.config.options['hessian_approximation'] = 'limited-memory' 
additional_options={'max_iter':3000}

for k,v in additional_options.items():
    solver.config.options[k] = v
results = solver.solve(mod, tee=True)

ValueError: invalid value for configuration 'nlp_solver':
	Failed casting cyipopt
	to <class 'pyomo.common.config.In'>
	Error: value cyipopt not in domain ['ipopt', 'gams', 'baron']